# Teste de Integração - Mini Dólar Strategy v2

Este notebook testa a integração entre os diferentes componentes da estratégia.

In [1]:
# Importações
import pandas as pd
import numpy as np
from datetime import datetime
import logging
import sys
from pathlib import Path
import os

# Adiciona o diretório raiz ao path
PROJECT_ROOT = Path(os.getcwd()).parent  # Subir um nível no diretório
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Importa os componentes da estratégia
from src.integrador.agent_base import BaseAgent, MarketData, TechnicalIndicators
from src.integrador.integration_manager import IntegrationManager
from src.relatorios.order_execution import OrderExecutionSystem
from src.relatorios.detailed_reporting import DetailedReportingSystem
from src.utils.logger import setup_logger

NameError: name 'AgentDecision' is not defined

## 1. Configuração do Logger

In [ ]:
logger = setup_logger("notebook_test")
logger.info("Iniciando testes de integração")

## 2. Carregamento dos Dados

In [ ]:
# Carrega dados históricos
def load_historical_data(filepath):
    try:
        data = pd.read_csv(filepath)
        logger.info(f"Dados carregados: {len(data)} registros")
        return data
    except Exception as e:
        logger.error(f"Erro ao carregar dados: {str(e)}")
        return None

# Substitua pelo caminho correto do seu arquivo de dados
historical_data = load_historical_data("../data/wdo_data.csv")
historical_data.head()

## 3. Inicialização dos Sistemas

In [ ]:
# Inicializa os componentes principais
integration_manager = IntegrationManager()
order_system = OrderExecutionSystem()
reporting_system = DetailedReportingSystem()

# Registra os agentes
agents = [
    RiskManagementAgent(max_drawdown=-0.02),
    MarketRegimeAgent(window_size=20),
    FeatureSelectionAgent(['sma_20', 'rsi', 'macd'])
]

for agent in agents:
    integration_manager.register_agent(agent)
    logger.info(f"Agente registrado: {agent.name}")

## 4. Função de Simulação

In [ ]:
def run_simulation(data, start_idx=0, periods=100):
    results = []
    
    for i in range(start_idx, min(start_idx + periods, len(data))):
        row = data.iloc[i]
        
        # Prepara dados de mercado
        market_data = MarketData(
            timestamp=pd.to_datetime(row['datetime']),
            open=row['open'],
            high=row['high'],
            low=row['low'],
            close=row['close'],
            volume=row['volume']
        )
        
        # Prepara indicadores técnicos
        technical_indicators = TechnicalIndicators(
            rsi=row['rsi'],
            macd={'line': row['macd'], 'signal': row['macd_signal'], 'histogram': row['macd_hist']},
            bollinger={'upper': row['bb_upper'], 'middle': row['bb_middle'], 'lower': row['bb_lower']},
            volume_profile={'poc': row['volume_poc']},
            additional={}
        )
        
        # Atualiza agentes e coleta decisões
        if integration_manager.update_market_data(market_data, technical_indicators):
            agent_decisions = integration_manager.collect_agent_decisions()
            
            # Consolida análises
            analysis = integration_manager.consolidate_analysis(
                agent_decisions,
                fundamental_data=None,
                ml_predictions=None
            )
            
            # Processa ordem se necessário
            if analysis.final_decision != 'HOLD':
                order = order_system.create_order(
                    analysis_result=analysis,
                    current_price=market_data.close
                )
                
                if order:
                    report = reporting_system.generate_detailed_trade_report(
                        order=order,
                        analysis_result=analysis
                    )
                    results.append({
                        'timestamp': market_data.timestamp,
                        'action': analysis.final_decision,
                        'price': market_data.close,
                        'order_id': order.order_id
                    })
        
        # Atualiza ordens existentes
        for active_order in order_system.get_active_orders():
            order_system.update_order(
                active_order.order_id,
                market_data.close,
                market_data.timestamp
            )
    
    return pd.DataFrame(results)


## 5. Execução do Teste

In [ ]:
# Executa simulação para os primeiros 100 períodos
results_df = run_simulation(historical_data, start_idx=0, periods=100)
results_df.head()

## 6. Análise dos Resultados

In [ ]:
# Gera relatório de performance
performance_report = order_system.generate_performance_report()

# Exibe métricas principais
print("\nMétricas de Performance:")
for key, value in performance_report['métricas_gerais'].items():
    print(f"{key}: {value}")

## 7. Visualização dos Resultados

In [ ]:
import matplotlib.pyplot as plt

# Plot dos preços e sinais
plt.figure(figsize=(15, 8))
plt.plot(historical_data['close'][:100], label='Preço')

# Marca sinais de compra e venda
for idx, row in results_df.iterrows():
    if row['action'] == 'CALL':
        plt.scatter(idx, row['price'], color='green', marker='^', s=100)
    elif row['action'] == 'PUT':
        plt.scatter(idx, row['price'], color='red', marker='v', s=100)

plt.title('Sinais de Trading')
plt.legend()
plt.grid(True)
plt.show()